<img src="Chicago.png">


# Selected Socioeconomic Indicators in Chicago

The city of Chicago released a dataset of socioeconomic data to the Chicago City Portal. This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

Scores on the hardship index can range from 1 to 100, with a higher index number representing a greater level of hardship.

A detailed description of the dataset can be found on the city of Chicago's website, but to summarize, the database has the following tables:

- census_data
- schools
- crimes


In [1]:
import sys
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
#import matplotlib.pyplot as plt

In [2]:
conn_string = "host='localhost' dbname='ChicagoDataPortal' user='postgres' password='dbpass'"

In [3]:
try:
    conn = psycopg2.connect(conn_string)
    print("Connection sucessful!")
except (Exception, psycopg2.Error) as error:
    print("Unable to connect to database")

Connection sucessful!


### Problem 1: Find the total number of crimes recorded in the CRIME table.

In [5]:
sql = ''' 
SELECT COUNT(*) 
FROM crimes;
'''
df  = sqlio.read_sql_query(sql, conn)
df.head()

,count
0,1000


### Problem 2: List community areas with per capita income less than 11000.

In [6]:
sql = ''' 
SELECT community_area_name 
FROM census_data 
WHERE per_capita_income_ < 11000;
'''
df  = sqlio.read_sql_query(sql, conn)
df

,community_area_name
0,West Garfield Park
1,South Lawndale
2,Fuller Park
3,Riverdale


### Problem 3: List all case numbers for crimes involving minors?

In [7]:
sql = ''' 
SELECT case_number 
FROM crimes  
WHERE description 
LIKE '%CHILD%'; 
'''
df  = sqlio.read_sql_query(sql, conn)
df

,case_number
0,JE261410
1,JE257480
2,JE256936
3,JE256370
4,JE255873


### Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [8]:
sql = ''' 
SELECT * 
FROM crimes 
WHERE UPPER(description) LIKE '%CHILD%' AND UPPER(primary_type) LIKE '%KID%' ;
'''
df  = sqlio.read_sql_query(sql, conn)
df.head()

,id,case_number,date_,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year_,updated_on,latitude,longitude,location_


### Problem 5: What kind of crimes were recorded at schools?

In [21]:
sql = ''' 
SELECT * 
FROM crimes 
WHERE UPPER(location_description) LIKE '%SCH%'; 
'''
df  = sqlio.read_sql_query(sql, conn)
df

,id,case_number,date_,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year_,updated_on,latitude,longitude,location_
0,12387644,JE257480,2021-06-07,013XX W 71ST ST,1582,OFFENSE INVOLVING CHILDREN,CHILD PORNOGRAPHY,SCHOOL - PUBLIC GROUNDS,False,False,...,6,67,17,1168443.0,1857678.0,2021,2021-06-14,41.764995,-87.658167,(41.76499529-87.65816697)
1,12389797,JE259304,2021-06-07,039XX N BELL AVE,820,THEFT,$500 AND UNDER,SCHOOL - PRIVATE GROUNDS,False,False,...,47,5,6,1160665.0,1926023.0,2021,2021-06-14,41.952705,-87.684783,(41.95270482-87.68478348)


### Problem 6: List the average safety score for all types of schools.

In [20]:
sql = ''' 
SELECT elementary_or_high_school, AVG(safety_score) 
FROM schools 
GROUP BY elementary_or_high_school;
'''
df  = sqlio.read_sql_query(sql, conn)
df

,elementary_or_high_school,avg
0,HS,49.623529
1,ES,49.520384
2,MS,48.000000


### Problem 7: List 5 community areas with highest % of households below poverty line.

In [22]:
sql = ''' 
SELECT community_area_name, percent_households_below_poverty 
FROM census_data 
ORDER BY percent_households_below_poverty DESC 
LIMIT(5);
'''
df  = sqlio.read_sql_query(sql, conn)
df

,community_area_name,percent_households_below_poverty
0,Riverdale,56.5
1,Fuller Park,51.2
2,Englewood,46.6
3,North Lawndale,43.1
4,East Garfield Park,42.4


### Problem 8: Which community area (number) is most crime prone?

In [27]:
sql = ''' 
SELECT community_area, COUNT(case_number) as number_of_cases 
FROM crimes 
GROUP BY community_area 
ORDER BY number_of_cases DESC
LIMIT(1);
 '''
df  = sqlio.read_sql_query(sql, conn)
df

,community_area,number_of_cases
0,25,54


### Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [28]:
sql = ''' 
SELECT ca, community_area_name
FROM census_data
WHERE hardship_index = (SELECT MAX(hardship_index) FROM census_data );
 '''
df  = sqlio.read_sql_query(sql, conn)
df

,ca,community_area_name
0,54,Riverdale


### Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [33]:
sql = '''
SELECT community_area, n_crimes, ca, community_area_name
FROM 
    ( SELECT community_area, COUNT(*) as n_crimes
      FROM crimes
      GROUP BY community_area )  as crimes_by_community
JOIN census_data 
ON census_data.ca = crimes_by_community.community_area 
ORDER BY n_crimes DESC
LIMIT(1);
'''
df  = sqlio.read_sql_query(sql, conn)
df

,community_area,n_crimes,ca,community_area_name
0,25,54,25,Austin
